# Radial Basis Function Neural Network

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from itertools import product

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt

from system_identification.rbfnn_model import RadialBasisFunctionNeuralNetworkModel

## 1D

In [3]:
def foo(x):
    return -x**2


inputs = np.linspace(-1, 1, 1000)
reference_outputs = np.array(list(map(foo, inputs)))
reference_outputs_noisy = reference_outputs + (np.random.random(reference_outputs.shape) - 0.5) * 0.1

inputs = inputs.reshape(-1, 1, 1)
reference_outputs = reference_outputs.reshape(-1, 1, 1)
reference_outputs_noisy = reference_outputs_noisy.reshape(-1, 1, 1)

### Least squares

In [4]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=1,
    grid_size=[9],
    input_range=np.array([(-1, 1)]),
    rbf_width=1.75,
    rbf_amplitude=1,
    log_dir="./"
)
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=1
 n_hidden=9
 n_outputs=1>

In [5]:
model.train(inputs, reference_outputs)

In [6]:
fig = plt.figure()
outputs = model.evaluate(inputs)
plt.plot(inputs.squeeze(), reference_outputs.squeeze())
plt.plot(inputs.squeeze(), reference_outputs_noisy.squeeze(), ".", markersize=1)
plt.plot(inputs.squeeze(), outputs.squeeze())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Levenberg-Marquardt training

In [7]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=1,
    grid_size=[3],
    input_range=np.array([(-1, 1)]),
    rbf_width=1.75,
    rbf_amplitude=1,
    log_dir="./"
)

In [8]:
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=1
 n_hidden=3
 n_outputs=1>

In [9]:
model.train(
    inputs,
    reference_outputs,
    epochs=1000,
    goal=1e-6,
    train_log_freq=1,
    method="trainlm",
    mu=10.,
    alpha=0.95
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [10]:
fig = plt.figure()
outputs = model.evaluate(inputs)
plt.plot(inputs.squeeze(), reference_outputs.squeeze())
plt.plot(inputs.squeeze(), reference_outputs_noisy.squeeze(), ".", markersize=1)
plt.plot(inputs.squeeze(), outputs.squeeze())

plt.figure()
model.training_log.error.plot()
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D

In [11]:
def foo(x):
    return -x[0]**2 + np.sin(x[1]*2)


resolution = 100
input_range = np.array([(-1, 1), (-3, 5)])
inputs = np.array(tuple(product(np.linspace(*input_range[0, :], resolution),
                                np.linspace(*input_range[1, :], resolution))))

reference_outputs = np.array(list(map(foo, inputs)))
reference_outputs_noisy = reference_outputs + (np.random.random(reference_outputs.shape) - 0.5) * 0.1

inputs = inputs[..., None]
reference_outputs = reference_outputs.reshape(-1, 1, 1)
reference_outputs_noisy = reference_outputs_noisy.reshape(-1, 1, 1)

### Least squares

In [12]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=2,
    grid_size=[5, 15],
    input_range=input_range,
    rbf_width=1.75,
    rbf_amplitude=1,
    log_dir="./"
)
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=2
 n_hidden=75
 n_outputs=1>

In [13]:
model.train(inputs, reference_outputs)
output = model.evaluate(inputs)

In [14]:
fig = plt.figure(figsize=(4, 3))
plt.plot(model.weights_c[:, 0], model.weights_c[:, 1], "x")

fig = plt.figure(figsize=(4, 3))
ax = fig.add_subplot(projection='3d')
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], output.squeeze(), s=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Levenberg-Marquardt training

In [15]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=2,
    grid_size=[5, 7],
    input_range=input_range,
    rbf_width=1.75,
    rbf_amplitude=1,
    log_dir="./"
)
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=2
 n_hidden=35
 n_outputs=1>

In [16]:
model.train(inputs, reference_outputs)
model.train(
    inputs,
    reference_outputs,
    epochs=1000,
    goal=1e-6,
    train_log_freq=1,
    method="trainlm",
    mu=10.,
    alpha=0.95
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [96]:
fig = plt.figure(figsize=(4, 3))
plt.plot(model.weights_c[:, 0], model.weights_c[:, 1], "x")

fig = plt.figure(figsize=(4, 3))
ax = fig.add_subplot(projection='3d')
output = model.evaluate(inputs)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], output.squeeze(), s=0.1)

plt.figure(figsize=(6, 3))
model.training_log.error.plot()
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
model_2 = model

In [93]:
plt.figure(figsize=(6, 3))
model_0.training_log.error.plot()
model_1.training_log.error.plot()
# model_2.training_log.error.plot()
model.training_log.error.plot()
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
input_range

array([[-1,  1],
       [-3,  5]])

In [29]:
RadialBasisFunctionNeuralNetworkModel.new_random_placement(
    n_inputs=2,
    n_hidden=4,
    input_range=input_range,
    rbf_width=1.75,
    rbf_amplitude=1,
    log_dir="./"
)

<RadialBasisFunctionNeuralNetwork
 n_inputs=2
 n_hidden=4
 n_outputs=1>

array([[2],
       [8]])